In [89]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pathlib import Path

from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [115]:
df = pd.read_csv(Path.home() / "Downloads/PA6_cleaned_dataset.csv")

## Pre Processing and Feature Engineering

In [144]:
def get_processed_df():
    df = pd.read_csv(Path.home() / "Downloads/PA6_cleaned_dataset.csv")
    df['time'] = pd.to_datetime(df['time'])
    df.dropna(subset="best_price_compound", inplace=True)
    df.fillna(method="ffill", inplace=True)

    # Perform seasonal decomposition
    result = seasonal_decompose(df['best_price_compound'], model='additive', period=12)  # Adjust period based on seasonality

    df['trend'] = result.trend
    df['seasonal'] = result.seasonal
    df['residual'] = result.resid

    # Encode date
    df['month'] = pd.to_datetime(df['time']).dt.month
    df['quarter'] = df['time'].dt.quarter
    df['year'] = df['time'].dt.year

    # Feature Engineering: Lagged Variables, as shown in CapGemini class
    df['best_price_compound_lag_1'] = df['best_price_compound'].shift(1)  # Lag of 1 month

    rolling_window = 3 
    df['rolling_mean'] = df['best_price_compound'].rolling(window=rolling_window).mean()
    df['rolling_std'] = df['best_price_compound'].rolling(window=rolling_window).std()

    # Feature Engineering: Seasonal Indicators
    df['is_winter'] = df['month'].isin([12, 1, 2]).astype(int)
    df['is_summer'] = df['month'].isin([6, 7, 8]).astype(int)
    df['is_spring'] = df['month'].isin([3, 4, 5]).astype(int)
    df['is_autumn'] = df['month'].isin([9, 10, 11]).astype(int)

    # Feature Engineering: Difference Features
    df['best_price_compound_diff_1'] = df['best_price_compound'] - df['best_price_compound'].shift(12)  # Assuming monthly data

    return df


def get_train_test_split():

    df = get_processed_df()
    df.fillna(method='backfill')
    df.set_index("time", inplace=True)
    train = df[df.index < '2022-01-01']
    test = df[df.index >= '2022-01-01']
    y_train = train["best_price_compound"]
    y_test = test["best_price_compound"]
    X_train = train.drop("best_price_compound", axis=1)
    X_test = test.drop("best_price_compound", axis=1)
    return X_train, X_test, y_train, y_test

In [140]:
df = get_processed_df()

## SARIMA training and testing

In [137]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

#df.set_index("time", inplace=True)
df.index.freq = 'MS'

train = df[df.index < '2022-01-01']
test = df[df.index >= '2022-01-01']
y_train = train["best_price_compound"]
y_test = test["best_price_compound"]
X_train = train.drop("best_price_compound", axis=1)
X_test = test.drop("best_price_compound", axis=1)

# Specify the order and seasonal_order based on your analysis
order = (1, 1, 1)  # Example order (p, d, q)
seasonal_order = (1, 1, 1, 12)  # Example seasonal_order (P, D, Q, s)

# Create and fit the SARIMAX model
sarimax_model = SARIMAX(endog=y_train, exog=X_train["residual"], order=order, seasonal_order=seasonal_order)
sarimax_result = sarimax_model.fit()

# Make predictions on the test set
predictions = sarimax_result.get_forecast(steps=len(test), exog=X_test["residual"])
predicted_values = predictions.predicted_mean

# Plot actual vs. predicted values
plt.figure(figsize=(12, 6))
plt.plot(train.index, y_train.values, label='Train')
plt.plot(test.index, y_test.values, label='Test')
plt.plot(test.index, predicted_values, label='Predicted', color='red')
plt.legend()
plt.show()


TypeError: Invalid comparison between dtype=int64 and str

### Not enough data for SARIMAX. Test with tree data

### XGB Model

Split train/test

In [73]:
X = df.drop(["best_price_compound", "time"], axis=1).copy()
y = df["best_price_compound"].copy()

X_train = df.query("time < '2021-01-01'").drop(["time", "best_price_compound"], axis=1)
X_test = df.query("time >= '2021-01-01'").drop(["time", "best_price_compound"], axis=1)

y_train = df.query("time < '2021-01-01'")["best_price_compound"]
y_test = df.query("time >= '2021-01-01'")["best_price_compound"]

Randomized search for hyperparameter thuning

In [78]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error


# Initialize an XGBoost model
model = XGBRegressor(random_state=42, n_jobs=-1)

# Define the hyperparameter grid for randomized search
param_dist = {
    'n_estimators': np.arange(100, 1000, 50),
    'max_depth': np.arange(3, 10),
    'learning_rate': np.logspace(-5, 0, num=100),
    'subsample': np.arange(0.5, 1.0, 0.1),
    'colsample_bytree': np.arange(0.5, 1.0, 0.1),
}

# Perform randomized search for hyperparameter tuning
random_search = RandomizedSearchCV(
    model, param_distributions=param_dist, n_iter=50, scoring='neg_mean_squared_error', cv=5, random_state=42, n_jobs=-1
)

# Fit the randomized search model on the training data
random_search.fit(X_train, y_train)

# Get the best hyperparameters from the search
best_params = random_search.best_params_

Compare two different XGB: one trained on tscv, the other trained on standard xtrain and xtest (is hp tuning still relevant if model is trained on tscv ?)

In [83]:
#
model_tscv = XGBRegressor(**best_params, random_state=42, n_jobs=-1)
model = XGBRegressor(**best_params, random_state=42, n_jobs=-1)

# Create a time series split iterator
tscv = TimeSeriesSplit(n_splits=5)

# Iterate through the splits and train the model
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the XGBoost model on the training data
    model_tscv.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred_ = model_tscv.predict(X_test)

    # Evaluate the model's performance
    mse = mean_squared_error(y_test, y_pred_)
    print(f'Mean Squared Error during tscv: {mse}')

model.fit(X_train, y_train)

# Evaluate on the test set
y_pred_tscv = model_tscv.predict(X_test)
mse_tscv = mean_squared_error(y_test, y_pred_tscv)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Squared Error on Test Set: {mse}')
print(f'Mean Squared Error on Test Set tcsv model: {mse_tscv}')

Mean Squared Error during tscv: 5555.752244937454
Mean Squared Error during tscv: 38942.48387119662
Mean Squared Error during tscv: 56452.10092933811
Mean Squared Error during tscv: 121577.02312411644
Mean Squared Error during tscv: 42438.160608338745
Best Hyperparameters: {'subsample': 0.8999999999999999, 'n_estimators': 550, 'max_depth': 5, 'learning_rate': 0.10974987654930568, 'colsample_bytree': 0.7999999999999999}
Mean Squared Error on Test Set: 42438.160608338745
Mean Squared Error on Test Set tcsv model: 42438.160608338745
